__Hoja de trabajo Proyecto BMW - Esther__

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


1.Importar dataset

In [134]:
df_bmw = pd.read_csv('dataset/bmw_pricing_v3.csv')

In [135]:
df_bmw.sample(5)

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
1302,BMW,520,165114.0,120.0,NaN,diesel,grey,estate,True,True,True,NaN,True,False,True,NaN,16300.0,2018-06-01
3196,BMW,520,251929.0,120.0,2013-04-01,diesel,black,sedan,True,True,False,False,True,False,True,True,16100.0,2018-05-01
623,BMW,318,155712.0,100.0,2013-07-01,diesel,blue,NaN,True,True,False,False,False,False,True,False,9500.0,2018-03-01
2334,BMW,116,88491.0,85.0,NaN,diesel,black,hatchback,False,False,False,NaN,False,NaN,True,False,10200.0,2018-07-01
3408,BMW,320,365452.0,120.0,2005-06-01,diesel,grey,sedan,True,False,False,NaN,False,NaN,True,False,3400.0,2018-07-01


In [136]:
df_bmw.shape

(4843, 18)

In [137]:
df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        3873 non-null   object 
 1   modelo                       4840 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               2420 non-null   object 
 5   tipo_gasolina                4838 non-null   object 
 6   color                        4398 non-null   object 
 7   tipo_coche                   3383 non-null   object 
 8   volante_regulable            4839 non-null   object 
 9   aire_acondicionado           4357 non-null   object 
 10  camara_trasera               4841 non-null   object 
 11  asientos_traseros_plegables  1452 non-null   object 
 12  elevalunas_electrico         4841 non-null   object 
 13  bluetooth         

Primeros comentarios:
1. Varias columnas con nulos
2. Campos fechas tipo 'object'
3. Solo el campo 'gps' es tipo booleano, en cambio campos como volante_regulable, aire_acondicionado, camara_trasera...alerta_lim_velocidad son 'objects' a pesar de tener valores 'True/False' 

In [138]:
#cambio las fechas a date_time
for i in ["fecha_registro","fecha_venta"]:
    df_bmw[i]=pd.to_datetime(df_bmw[i])

In [139]:
df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   marca                        3873 non-null   object        
 1   modelo                       4840 non-null   object        
 2   km                           4841 non-null   float64       
 3   potencia                     4842 non-null   float64       
 4   fecha_registro               2420 non-null   datetime64[ns]
 5   tipo_gasolina                4838 non-null   object        
 6   color                        4398 non-null   object        
 7   tipo_coche                   3383 non-null   object        
 8   volante_regulable            4839 non-null   object        
 9   aire_acondicionado           4357 non-null   object        
 10  camara_trasera               4841 non-null   object        
 11  asientos_traseros_plegables  1452 non-null 

2. Analizamos duplicados

In [140]:
df_bmw[df_bmw.duplicated()].shape

(0, 18)

Comentarios: sin duplicados

3 .Analizar nulos


In [141]:
df_bmw.isnull().sum()

marca                           970
modelo                            3
km                                2
potencia                          1
fecha_registro                 2423
tipo_gasolina                     5
color                           445
tipo_coche                     1460
volante_regulable                 4
aire_acondicionado              486
camara_trasera                    2
asientos_traseros_plegables    3391
elevalunas_electrico              2
bluetooth                       728
gps                               0
alerta_lim_velocidad            728
precio                            6
fecha_venta                       1
dtype: int64

3.1. Marca: vemos que solo tiene dos valores 'BMW' y 'nan'. Por lo tanto como no aporta información, la borramos (haciendo una copia antes)


In [142]:
df_bmw['marca'].unique()

array([nan, 'BMW'], dtype=object)

In [143]:
df_bmw1 =  df_bmw.copy()

In [144]:
del(df_bmw1['marca'])

In [ ]:
df_bmw1.info()

3.2 Analizamos las categorías con muy pocos nulos: modelo, km, potencia, tipo_gasolina, volante_regulable, camara_trasera, elevalunas_electrico, precio, fecha_venta

In [156]:
col_few_nulls = ['modelo', 'km', 'potencia', 'tipo_gasolina', 'volante_regulable', 'camara_trasera', 'elevalunas_electrico', 'precio', 'fecha_venta']

In [157]:
num_rows_nulls = len(df_bmw1[df_bmw1['modelo'].isnull() | df_bmw1['km'].isnull() | df_bmw1['tipo_gasolina'].isnull() | df_bmw1['volante_regulable'].isnull() | df_bmw1['camara_trasera'].isnull() | df_bmw1['elevalunas_electrico'].isnull() | df_bmw1['precio'].isnull() | df_bmw1['fecha_venta'].isnull()]) 
print(num_rows_nulls)
print(num_rows_nulls / df_bmw1.shape[0] * 100)

25
0.5162089613875697


Al ser tan pocas filas las que tienen alguno de estos campos a nulo, las elimino

In [158]:
df_bmw2 = df_bmw1.copy()

In [159]:
df_bmw2.dropna(subset = col_few_nulls, inplace=True)

In [161]:
df_bmw.shape[0] - df_bmw2.shape[0]

26